In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

import implicit
import lightfm
import warnings
from validation_prepare import *
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns',100)

DATA_PATH = '../okko/orig_data'
PREPARED_PATH = './prepared_data/'

In [3]:
actions = pd.read_pickle(PREPARED_PATH+'actions_one_table.pkl')

actions.sort_index(inplace = True) # На всякий случай, иначе деление не будет работать

actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [4]:
idx = get_train_test(actions)

6558458
2186152
2186153


In [5]:
actions.iloc[idx[0]].index.get_level_values(2).max()

43362401.96226887

In [6]:
actions.iloc[idx[1]].index.get_level_values(2).min()

43362401.97085199

In [7]:
actions.iloc[idx[1]].index.get_level_values(2).max()

43828341.47903843

In [8]:
actions.iloc[idx[2]].index.get_level_values(2).min()

43828341.48519237

In [9]:
actions.consumption_mode.value_counts()

S    8296227
P     873834
R     472951
Name: consumption_mode, dtype: int64

In [10]:
actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [11]:
# Вроде не пересекается.
train,test,valid = actions.iloc[idx[0]],actions.iloc[idx[1]],actions.iloc[idx[2]]

In [12]:
%time 
dur_being_train,dur_films_train,watch_actions_train,target_train = get_target(train)
dur_being_test,dur_films_test,watch_actions_test,target_test = get_target(test)
dur_being_valid,dur_films_valid,watch_actions_valid,target_valid = get_target(valid)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs


In [13]:
# Получили фичи для фильмов
import pickle
with open(PREPARED_PATH+'catalogue_features.pkl','rb') as f:
    match_element_row,match_row_element,match_columns,element_matrix = pickle.load(f)
movie_match_columns = {i:ii for ii,i in enumerate(match_columns)}
movie_columns_match = {ii:i for ii,i in enumerate(match_columns)}

with open(PREPARED_PATH+'bag_of_attr_movie.pkl','rb') as f:
    bag_of_attr = pickle.load(f)

In [14]:
fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
                       'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
                      'movie_match_movie_row':match_element_row,},bag_of_attr,)
fe.fit(train)

100%|██████████| 389188/389188 [03:30<00:00, 1848.38it/s]


(389188, 29221) 389188 29221


In [15]:
train_ = fe.transform(train)
test_ = fe.transform(test)

train_.shape,test_.shape 

cfe = ColdFeatureExtractor(fe)

cfe.fit(train)

max(cfe.train_movie_cols)

# sp.csc_matrix(cfe.fitted_FE.movie_attr_matrix)[cfe.train_movie_rows,cfe.train_movie_cols]



train_res = cfe.transform(test)


print(train_res['train_user'].shape,train_res['test_user'].shape,train_res['new_test_user'].shape)

print(train_res['train_movie'].shape,train_res['test_movie'].shape)

8015 389188
(286374, 1) (281788, 3)
(389188, 8015)
8015 389188
(85260, 1) (63113, 3)
(389188, 8015)


100%|██████████| 35799/35799 [00:14<00:00, 2444.18it/s]


(389188, 29221) (389188, 29221) (6654, 29221)
(8015, 29232) (251, 29232)


In [16]:
'''
Здесь начинается русское поле экспериментов над параметрами моделей.
'''

'\nЗдесь начинается русское поле экспериментов над параметрами моделей.\n'

In [17]:
'''
Здесь заканчиваетс/ русское поле экспериментов
'''

'\nЗдесь заканчиваетс/ русское поле экспериментов\n'

In [18]:
actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [19]:
_,_,watch_actions,_ = get_target(actions)

In [20]:
actions = actions.join(watch_actions['rel_dur'])

In [21]:
actions['rel_dur'] = actions['rel_dur'].fillna(0).replace(np.inf,1)

In [22]:
#Проверим, что не загибается на полном датасете

fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
                       'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
                      'movie_match_movie_row':match_element_row,},bag_of_attr,mode = 'duration',delimiter = 1/3,
                     target_col_name = 'rel_dur',)


# fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
#                        'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
#                       'movie_match_movie_row':match_element_row,},bag_of_attr,is_censor = False)
X = fe.fit_transform(actions)

100%|██████████| 499663/499663 [04:29<00:00, 1855.99it/s]


(499663, 29988) 499663 29988
8296 499663
(9643012, 1) (9643012, 3)
(499663, 8296)


In [23]:
cfe = ColdFeatureExtractor(fe)

res_action = cfe.fit_transform(actions)

100%|██████████| 499663/499663 [05:05<00:00, 1636.33it/s]


In [26]:
res_action.keys()

dict_keys(['train_user', 'test_user', 'new_test_user', 'map_user_row', 'map_row_user', 'map_user_row_new', 'map_row_user_new', 'map_attr_column', 'map_column_attr', 'train_movie', 'test_movie', 'map_movie_row', 'map_row_movie', 'map_movie_row_new', 'map_row_movie_new', 'map_feature_column', 'map_column_feature'])

In [24]:
len(fe.train_movie_match_movie_row),len(fe.match_user_row)

(8296, 499663)

In [25]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

In [27]:
X_user = res_action['train_user']
X_movie = res_action['train_movie']
X_user.shape,X_movie.shape

((499663, 29988), (8296, 29999))

In [28]:
# def fit_lightfm(train,item_features=None,seed = 0)
seed = 0
epochs = 30
num_threads=4

model = lightfm.LightFM(loss = 'warp',random_state=seed)


# model.fit(train_matrix,user_features = train_user,item_features = element_matrix,epochs = epochs,num_threads = num_threads,
#          verbose = True)


In [30]:


%time model.fit(X,user_features=X_user,item_features=X_movie,epochs = epochs,num_threads = num_threads,verbose = True)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
CPU times: user 47.7 s, sys: 766 ms, total: 48.5 s
Wall time: 23.7 s


In [35]:
%%time
a,b = X.nonzero()
D = pd.DataFrame([a,b],index = ['row','columns']).T
D = D.groupby('row').agg(lambda x:frozenset(x.values))
D

CPU times: user 8min 10s, sys: 3.31 s, total: 8min 13s
Wall time: 8min 13s


In [36]:
D.loc[1]

columns    (5537, 354, 1347, 7650, 5638, 4583, 6568, 1801...
Name: 1, dtype: object

In [37]:
len(fe.train_movie_match_row_movie)

8296

In [38]:
print('sparsity',X.nnz/(X.shape[0]*X.shape[1]))

sparsity 0.002326305634915966


In [39]:
len( set(test_users) - set(fe.match_user_row.keys()))

8

In [40]:
from implicit.nearest_neighbours import TFIDFRecommender,CosineRecommender,NearestNeighboursScorer



In [41]:
# Для implicit  надо фильмы*юзеры матрицу.
# model = CosineRecommender()
# model.fit(X.T.tocsr())

In [42]:
#ratings_matrix_T.shape

NameError: name 'ratings_matrix_T' is not defined

In [ ]:
# Сделаем какой-нибудь сабмит из lightfm
result = {}
mode = 'lightfm'#'implicit'#'lightfm'
cnt = 0
for user_uid in tqdm.tqdm(test_users):
    
    # transform user_uid to model's internal user category
    try:
        user_cat = fe.match_user_row[user_uid]
    except LookupError:
        continue
    if mode == 'lightfm':
        need_cols = np.array(list(set(np.arange(X.shape[1])) - D.get(user_cat, set())))
        recs = model.predict(user_cat,need_cols,user_features=X_user,item_features=X_movie,num_threads=num_threads)
        need_movies = np.argsort(-recs)[:20]
        result[user_uid] = [int(fe.train_movie_match_row_movie[i]) for i in need_movies]
    # переводим в фильмы
    
    # perform inference
    if mode == 'implicit':
        if cnt == 0:
            ratings_matrix_T = X.T.tocsr()
        recs = model.recommend(
            user_cat,
            X.tocsr(),
            N=20,
            filter_already_liked_items=True,
            filter_items=D.get(user_uid, set())
        )
        result[user_uid] = [int(fe.train_movie_match_row_movie[i]) for i, _ in recs]
        # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    cnt += 1

 30%|███       | 15045/50000 [23:23<50:21, 11.57it/s]  

In [ ]:
user_cat

In [ ]:
len(fe.train_movie_match_row_movie)

In [ ]:
recs

In [ ]:
#len(need_cols),len(recs),type(recs)

In [ ]:
#plt.hist(recs)

In [ ]:
len(result)

In [ ]:

with open('answer.json', 'w') as f:
    json.dump(result, f)

In [ ]:
te.shape

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, te ,tr, k = 20,num_threads=4).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
new_old_tr_te_2.keys()

In [ ]:
tr = new_old_tr_te_2['train_interactions']
te = new_old_tr_te_2['test_interactions']
uf = new_old_tr_te_2['user_features_train']
tf = new_old_tr_te_2['user_features_test']
print(tr.shape,te.shape,uf.shape,tf.shape)
# tr[(tr < 3) &  (tr>0)] = -1
# tr[(tr > 3)] = 1
# te[(tr < 3) &  (te>0)] = -1
# te[(tr > 3)] = 1

model.fit(train_matrix,user_features=uf,epochs = epochs,num_threads =num_threads,
         verbose = True)

In [ ]:
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt


In [ ]:
p = model.predict(0,np.arange(tr.shape[1]),user_features=tf[0,:],num_threads=num_threads)

In [ ]:
plt.hist(p)

In [ ]:
test_user[0,:]

In [ ]:
a = model.predict(0,np.arange(train_matrix.shape[1]),user_features=test_user[0,:],num_threads=num_threads)
len(a)

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, train_matrix,user_features = train_user,item_features = element_matrix ,num_threads=num_threads).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
model.user_feature_map

In [ ]:
def get_answer(test_users_dict,test):
    pass